## Averaging attention distributions 6, 7, and 8 using a GRU 

We extract the attention distribution directly from a model that has been pretrained on bias detection!

We average together the 6th, 7th, and 8th attention distributions of the input. Our classification model is an LSTM that reads in the attention distribution for each word sequentially. In general the max sequence lenght is 80 which means that our model will read in 80 data points which are each 80 dimensional. 


#### Notes
* One remaining question is how can we experiment with different number of attention heads. In general the extract attention scores function seems to have some bugs that need ironing out - such as only being able to pass in a batch size of 1 into the extraction schema.

In [2]:
import sys; sys.path.append("../../../../..")
import torch 
from src.experiment import AttentionExperiment, ClassificationExperiment
from src.dataset import ExperimentDataset
from src.params import Params
from src.utils.attention_utils import reduce_attention_dist
from src.utils.classification_utils import run_bootstrapping

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
params = Params.read_params("experiment_params.json")
print("layers = {}".format(params.intermediary_task["attention"]["layers"]))
print("reducer = {}".format(params.intermediary_task["attention"]["reducer"]))

layers = [6, 7, 8]
reducer = avg


In [5]:
# Loading in the dataset that we are using in this experiments 
# typically this dataset is the small set of ground-truth labels
dataset = ExperimentDataset.init_dataset(params.dataset)
dataset

03/13/2020 10:23:34 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
386it [00:00, 4846.71it/s]


Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label'])

Attention Experiment: 
* Is a class that wraps useful methods to extract attention distributions from a given BERT-based model 
* The user has to provide in two config files: One to specify parameters for how the attention scores should be extracted and combined, and other to specify the intermediary model from which the attention scores should be extracted from
* The user needs to instantiate the attention experiment with a function that tells the model how to run 
 inference on the given model. The function header is specified below: 
 
 ``` def initialize_attention_experiment(cls, intermediary_task_params, dataset_params, verbose=False) ```
 


In [6]:
attention_dataloader = dataset.return_dataloader() 
attention_experiment = AttentionExperiment.initialize_attention_experiment(params.intermediary_task, params.dataset, verbose=True)

03/13/2020 10:23:36 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at ./cache/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


The len of our vocabulary is 30523
Cuda is set to true


03/13/2020 10:23:36 - INFO - pytorch_pretrained_bert.modeling -   loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
03/13/2020 10:23:36 - INFO - pytorch_pretrained_bert.modeling -   extracting archive file ./cache/9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir /tmp/tmpn1d86_5w
03/13/2020 10:23:44 - INFO - pytorch_pretrained_bert.modeling -   Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

03/13/2020 10:23:52 - INFO - pytor

Instantiated joint model with pretrained weights.
Succesfully loaded in attention experiment!


```extract_attention_scores()``` works out of the box because the attention experiment has the config file saved, and knows what BERT model to use/load in, which layers to extract the attention scores from, and what the inference function is that should be used on this particular BERT model.

Attention_scores is then a list of dictionaries. The keys in this dictionary are the specific layers of a BERT model and the values are the corresponding attention distributions extracted from that particular layer.

In [7]:
attention_scores = attention_experiment.extract_attention_scores(attention_dataloader)

In [8]:
reduced_attention = reduce_attention_dist(attention_scores, params.intermediary_task["attention"]["reducer"])
stacked_reduced_attention = torch.stack(reduced_attention).squeeze()
dataset.add_data(stacked_reduced_attention, "attention_dist")
dataset.shuffle_data()
print("reduced_attention[0].shape = {}".format(reduced_attention[0].shape))
print("stacked_reduced_attention.shape = {}".format(stacked_reduced_attention.shape))
dataset

reduced_attention[0].shape = torch.Size([1, 1, 80, 80])
stacked_reduced_attention.shape = torch.Size([324, 80, 80])


Length: 324 Keys: dict_keys(['pre_ids', 'masks', 'pre_lens', 'post_in_ids', 'post_out_ids', 'pre_tok_label_ids', 'post_tok_label_ids', 'rel_ids', 'pos_ids', 'categories', 'index', 'bias_label', 'attention_dist'])

### This is where the classification experiment starts

We create a classification experiment that contains useful methods for classifying bias based on the attention distributions. 

In [9]:
classification_experiment = ClassificationExperiment.init_cls_experiment(params.final_task)

In [ ]:
stats = run_bootstrapping(classification_experiment, dataset, params.final_task, num_bootstrap_iters=5, input_key="attention_dist", label_key="bias_label")

In [11]:
stats

{'auc': [(0.5296318476844792, 0.5887528491916246), 0.5608006354547708],
 'accuracy': [(0.6441964285714287, 0.6860969387755101), 0.6632653061224489]}

In [ ]:
classification_experiment.save_model_weights("gru-attention.weights")